In [150]:
import re
import pandas as pd
from seqeval.scheme import IOB2
from seqeval.metrics import classification_report as cr
from seqeval.metrics import performance_measure
from seqeval.scheme import IOB2
from seqeval.metrics import performance_measure
import conllu
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score, precision_score, recall_score, classification_report
from torch.utils.data import Dataset
import torch
import numpy as np
import os




with open("/Users/aarnes/Documents/GitHub/MA_Peter-R-ysland-Aarnes/Results/NCRFpp/prediction_norne_ncrf.txt", "r") as f:
    preds_norne = f.readlines()
with open("/Users/aarnes/Documents/GitHub/MA_Peter-R-ysland-Aarnes/Results/NCRFpp/prediction_parl_ncrf.txt", "r") as f:
    preds_parl = f.readlines()


In [151]:
preds_norne = [re.sub('\n', '', line) for line in preds_norne]
preds_parl = [re.sub('\n', '', line) for line in preds_parl]


In [152]:
# Heavily insipired from:
# https://github.com/ltgoslo/NorBERT/blob/main/benchmarking/experiments/dataset.py

class CoNLLDataset(Dataset):
    def __init__(self, x_tokens, y_labels, ner_vocab=None):
        self.tokens = [[x for x in entry] for entry in x_tokens]
        self.ner_labels = [[y for y in entry] for entry in y_labels]

        # hard coded ner_vocab to avoid random shuffled instanciation of ordering of ner_vocab
        self.ner_vocab = ner_vocab
        self.ner_indexer = {i: n for n, i in enumerate(self.ner_vocab)}
    
    def __getitem__(self, index):
        tokens = self.tokens[index]
        ner_labels = self.ner_labels[index]

        x = tokens
        y = torch.LongTensor([self.ner_indexer[i] if i in self.ner_vocab
                              else self.ner_indexer['@UNK'] for i in ner_labels])
        return x, y

    def __len__(self):
        return len(self.tokens)



In [153]:
import json

class ParlamentaryCorpus():

    def __init__(self, path): # Takes input normalized_token_file
        self.path = path
        self.sentence_dict = {}

    def load_data(self, lower=False):
        sentence_dict = self.sentence_dict
        with open(self.path) as infile:
            with open(self.path, encoding="UTF-8") as infile:
                json_input = json.load(infile)

        if lower == False:
            for chaos in json_input["sentences"]:
                token_list = []
                for token in chaos["tokens"]:
                    if token["special_status"] != None:
                        continue
                    else:
                        if " " in token["token_text"]:
                            token_list.extend(token["token_text"].split())
                        else:
                            token_list.append(token["token_text"])
                    sentence_dict[chaos["sentence_id"]] = token_list
        else:
            for chaos in json_input["sentences"]:
                token_list = []
                for token in chaos["tokens"]:
                    if token["special_status"] != None:
                        continue
                    else:
                        if " " in token["token_text"]:
                            token_list.extend(token["token_text"].lower().split())
                        else:
                            token_list.append(token["token_text"].lower())
                    sentence_dict[chaos["sentence_id"]] = token_list
                    
        return sentence_dict

In [154]:
## Conllu stuff
def filter_tags(x):
    return x        

def convert_to_list_dict(path, file):
    path = path.format(file)
    with open(path, encoding="UTF-8") as infile:
        lst = []
        tokens = list(conllu.parse_incr(infile))
        for sent in tokens:
                dic = {
                "idx": sent.metadata["sent_id"],
                "text": sent.metadata["text"].lower(),
                "tokens": [token["form"].lower() for token in sent],
                "lemmas": [token["lemma"] for token in sent],
                "pos_tags": [token["upos"] for token in sent],
                "ner_tags": [filter_tags(token["misc"].get("name", "O")) for token in sent],
            }
                lst.append(dic) 
        print("Converting {} to list of dictionaries\n     {} elements converted..".format(file, len(lst)))
    return lst

In [155]:
path = "all_conllu/{0}.conllu"
file_list = ["no_bokmaal-ud-dev", "no_bokmaal-ud-test", "no_bokmaal-ud-train", "no_nynorsk-ud-dev", "no_nynorsk-ud-test", "no_nynorsk-ud-train"]
test_split_no = convert_to_list_dict(path, file_list[1])
test_split_ny = convert_to_list_dict(path, file_list[4])
test_split = test_split_no + test_split_ny


Converting no_bokmaal-ud-test to list of dictionaries
     1939 elements converted..
Converting no_nynorsk-ud-test to list of dictionaries
     1511 elements converted..


In [156]:
ner_vocab =     ['B-GPE_LOC', 'I-DRV', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 
                'I-GPE_ORG', 'B-GPE_ORG', 'B-EVT', 'B-DRV', 'I-PROD', 'B-ORG', 'B-MISC',
                'I-MISC', 'I-GPE_LOC', 'B-LOC', 'I-ORG', 'I-EVT', 'O', '@UNK']

x_test_tokens = [x["tokens"] for x in test_split]
y_test_labels = [y["ner_tags"] for y in test_split]
test_dataset = CoNLLDataset(x_test_tokens, y_test_labels, ner_vocab)

In [157]:
gold_labels = []
for sentence_labels in test_dataset.ner_labels:
    for label in sentence_labels:
        gold_labels.append(label)

In [158]:
"""Making list of lists preds, same len as NorNE gold labels"""
""" test_dataset_ner_labels should be test_dataset.ner_labels """

def split_list_preds(preds, test_dataset_ner_labels):
    split_list_preds = []
    start = 0

    for sublist in test_dataset_ner_labels:
        end = start + len(sublist)
        split_list_preds.append(preds[start:end])
        start = end
    return split_list_preds

In [159]:
def transform_to_binary(set):
    binary_labels = []
    for label in set:
        if label == "O":
            binary_labels.append(0)
        else:
            binary_labels.append(1)
    return binary_labels

In [160]:
def gold_labels_unlabled_data(unlabeled):
    length = 0
    gold_labels = []
    for sentences in unlabeled:
        label = []
        length += len(sentences)
        for token in sentences:
            if token[0].isupper() == True: # Checks if token has captial letter
                label.append(1)
                
            else:
                label.append(0) # Adds 0 if token has lower 
        gold_labels.append(label)
    
    return gold_labels

In [161]:
def parl_sentences_only(dictionary_corpus):
    corp = []
    for diction in dictionary_corpus:
        corp += list(diction.values())
    return corp

def load_parl_corpus(rootdir_parl_corpus, lower=False):
    corpora_normal_cap, corpora_lower, paths = [], [], []
    
    for subdir, dirs, files in os.walk(rootdir_parl_corpus):
        for file in files:
            if "normalized_token_data.json" in file:
                path = (os.path.join(subdir, file))
                paths.append(path)

    for corpus, path in enumerate(paths):
        corpus = ParlamentaryCorpus(path)
        corpus = corpus.load_data()
        corpora_normal_cap.append(corpus)
        for k, v in corpus.items():
            if v == []:
                print(k)
                print(path)

    if lower==True:
        for corpus, path in enumerate(paths):
            corpus = ParlamentaryCorpus(path)
            corpus = corpus.load_data(lower=True)
            corpora_lower.append(corpus)


    return corpora_normal_cap, corpora_lower

In [162]:
""" Reults section """

' Reults section '

In [163]:
rootdir=r"/Users/aarnes/Documents/GitHub/MA_Peter-R-ysland-Aarnes/parl_corpus_full"
corpora_normal_cap, corpora_lower = load_parl_corpus(rootdir, lower=True)
gold_corp = parl_sentences_only(corpora_normal_cap)
_gold_labels_parl = gold_labels_unlabled_data(gold_corp)
gold_labels_parl = [item for sublist  in _gold_labels_parl for item in sublist]

In [164]:
norne_preds_binary = transform_to_binary(preds_norne)
parl_preds_binary = transform_to_binary(preds_parl)
gold_binary = transform_to_binary(gold_labels)
y_pred_parl_binary = transform_to_binary(preds_parl)



In [165]:
preds_split_sentences = split_list_preds(preds_norne, test_dataset.ner_labels)

In [166]:
# CLASSIFICATION REPORT
norne_cr_df_all_labels = classification_report(gold_labels, preds_norne, labels = ner_vocab[:-2], digits=5, output_dict=True)
norne_cr_IOB2 = cr(test_dataset.ner_labels, preds_split_sentences, mode='strict', scheme=IOB2, digits=4, output_dict=True)
norne_cr_binary = classification_report(gold_binary, norne_preds_binary, digits=5, output_dict=True)

norne_cr_df_all_labels = pd.DataFrame(norne_cr_df_all_labels).transpose()
norne_cr_df_IOB2 = pd.DataFrame(norne_cr_IOB2).transpose()
norne_cr_df_binary = pd.DataFrame(norne_cr_binary).transpose()


## CONFUSION MATRIX
norne_cf_matrix_no_O = confusion_matrix(gold_labels, preds_norne, labels = ner_vocab[:-2])
norne_cf_matrix_O = confusion_matrix(gold_labels, preds_norne, labels = ner_vocab[:-1])
norne_cr_matrix_binary = confusion_matrix(gold_binary, norne_preds_binary)

norne_cf_df_no_O = pd.DataFrame(norne_cf_matrix_no_O)
norne_cf_df_O = pd.DataFrame(norne_cf_matrix_O)
norne_cf_df_binary = pd.DataFrame(norne_cr_matrix_binary)

norne_cf_df_no_O.columns = ner_vocab[:-2]
norne_cf_df_no_O.index = ner_vocab[:-2]
norne_cf_df_O.columns = ner_vocab[:-1]
norne_cf_df_O.index = ner_vocab[:-1]


# class report
norne_cr_df_all_labels.to_csv("norne_cr_df_all_labels", mode="a")
norne_cr_df_IOB2.to_csv("norne_cr_df_IOB2", mode="a")
norne_cr_df_binary.to_csv("norne_cr_df_binary", mode="a")

#confusion matrix
norne_cf_df_no_O.to_csv("norne_cf_df_no_O", mode="a")
norne_cf_df_O.to_csv("norne_cf_df_O", mode="a")
norne_cf_df_binary.to_csv("norne_cf_df_binary", mode="a")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [ ]:
parl_cr = classification_report(gold_labels_parl, y_pred_parl_binary, digits=5, output_dict=True)
parl_cf_matrix = confusion_matrix(gold_labels_parl, y_pred_parl_binary)

parl_cr_df = pd.DataFrame(parl_cr).transpose()
parl_cf_df = pd.DataFrame(parl_cf_matrix)

parl_cf_df.columns = [0, 1]
parl_cf_df.index = [0, 1]

parl_cr_df = pd.DataFrame(parl_cr).transpose()
parl_cf_df = pd.DataFrame(parl_cf_matrix)

parl_cf_df.columns = [0, 1]
parl_cf_df.index = [0, 1]

parl_cr_df.to_csv("parl_classification_report_ncrf", mode="a")
parl_cf_df.to_csv("parl_matrix_ncrf", mode="a")

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

[]